## Samhitha Gundam
### HW 2 -
**Question 1,2,3,4 are present in this notebook and Q5 is present in the second Notebook**

In [1]:
!pip install torch

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import gensim.models
import gensim.downloader as api
import nltk
import numpy as np
nltk.download('punkt')
import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler

[nltk_data] Downloading package punkt to /Users/samhitha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Question 1**

In [3]:
raw_data=pd.read_csv("amazon_reviews_us_Jewelry_v1_00.tsv",sep="\t",on_bad_lines='skip')
data=raw_data[['star_rating','review_body']]
data=data.dropna()
data = data.reset_index(drop=True)
data['star_rating']=data['star_rating'].astype(int)

/var/folders/3c/dvx5c3n50kx5fh6qcgyy221w0000gn/T/ipykernel_3452/4051224240.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data=pd.read_csv("amazon_reviews_us_Jewelry_v1_00.tsv",sep="\t",on_bad_lines='skip')


In [4]:
data

,star_rating,review_body
0,5,so beautiful even tho clearly not high end ......
1,5,"Great product.. I got this set for my mother, ..."
2,5,Exactly as pictured and my daughter's friend l...
3,5,Love it. Fits great. Super comfortable and nea...
4,5,Got this as a Mother's Day gift for my Mom and...
...,...,...
1766743,4,It is nice looking and everything (it is sterl...
1766744,4,"my boyfriend bought me this last christmas, an..."
1766745,4,This is a great way to quickly start learning ...
1766746,5,the 14kt gold earrings look remarkable...would...


In [4]:
df=data.groupby('star_rating').sample(n=20000)

In [5]:
df['review_body']=df['review_body'].str.lower()  #convert to lower case

#remove contractions
df['review_body']=df['review_body'].str.replace("\'re"," are")
df['review_body']=df['review_body'].str.replace("br"," ")
df['review_body']=df['review_body'].str.replace("\n't"," not")
df['review_body']=df['review_body'].str.replace("\'s'"," is")
df['review_body']=df['review_body'].str.replace("i'm"," i am")
df['review_body']=df['review_body'].str.replace("\'ve'"," have")
df['review_body']=df['review_body'].str.replace("din't","did not")

df['review_body']=df['review_body'].str.replace('http\S+|www.\S+', '', case=False)
df['review_body']=df['review_body'].str.replace('[^a-zA-Z0-9 ]', '')
df['review_body']=df['review_body'].str.replace('/ +/', ' ')#convert multispace to space
df['review_body']=df['review_body'].str.replace('/^ /', '') # remove spaces in the start of the string
df['review_body']=df['review_body'].str.replace('/ $/', '') # remove unnecesary space at the end of the string



/var/folders/3c/dvx5c3n50kx5fh6qcgyy221w0000gn/T/ipykernel_3452/77415047.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review_body']=df['review_body'].str.replace('http\S+|www.\S+', '', case=False)
/var/folders/3c/dvx5c3n50kx5fh6qcgyy221w0000gn/T/ipykernel_3452/77415047.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review_body']=df['review_body'].str.replace('[^a-zA-Z0-9 ]', '')
/var/folders/3c/dvx5c3n50kx5fh6qcgyy221w0000gn/T/ipykernel_3452/77415047.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review_body']=df['review_body'].str.replace('/ +/', ' ')#convert multispace to space
/var/folders/3c/dvx5c3n50kx5fh6qcgyy221w0000gn/T/ipykernel_3452/77415047.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review_body']=df['review_body'].str.replace('/^ /',

**Question 2(a)** Implementation of word2Vec using google new data set

In [6]:
wv = api.load('word2vec-google-news-300')

In [7]:
wv.most_similar(positive = ['king','woman'], negative =['man'],topn=3)

[('queen', 0.7118193507194519),
 ('monarch', 0.6189674735069275),
 ('princess', 0.5902431011199951)]

In [9]:
wv.most_similar(positive = ['father','woman'], negative =['man'],topn=3)

[('mother', 0.8462508320808411),
 ('daughter', 0.7899606227874756),
 ('husband', 0.7560456991195679)]

In [10]:
print('excellent', 'outstanding',wv.similarity('excellent','outstanding'))

excellent outstanding 0.5567486


In [12]:
print('good', 'great',wv.similarity('good','great'))
print('worst','bad',wv.similarity('worst','bad'))
print('car', 'truck',wv.similarity('car','truck'))

good great 0.72915095
worst bad 0.43674564
car truck 0.67357904


**Question 2b** - Word2Vec using review dataset

In [13]:
X=df['review_body']
Y=df['star_rating']

In [14]:
inputs=X.to_list()
sentences=[k.split() for k in inputs]

In [15]:
model = gensim.models.Word2Vec(sentences=sentences,vector_size=300, window=11, min_count=10)

In [16]:
model.wv.most_similar(positive = ['king','woman'], negative =['man'],topn=3)

[('avenue', 0.5949714779853821),
 ('candy', 0.5309351086616516),
 ('amazoncom', 0.5016636252403259)]

In [19]:
model.wv.most_similar(positive = ['father','woman'], negative =['man'],topn=3)

[('grandchildren', 0.6483133435249329),
 ('lady', 0.6390446424484253),
 ('cousin', 0.6286104321479797)]

In [20]:
print('excellent', 'outstanding',model.wv.similarity('excellent','outstanding'))
print('good', 'great',model.wv.similarity('good','great'))
print('worst','bad',model.wv.similarity('good','great'))

excellent outstanding 0.70434016
good great 0.83079803
worst bad 0.83079803


**Using the above values of similarity we can state that pretrained google model is better for terms like king, queen but for case of adjectives like good, bad, excellent which are greatly a part of the review dataset, our word2vec model performs better as there are greater examples with such used thus allowing the model to be trained better**

**Question 3**

In [21]:
X=df['review_body']
Y=df['star_rating']

In [22]:
X=pd.DataFrame(X)
sentences=[k.split() for k in X['review_body']]
X['reviews_split']=sentences

In [23]:
X

,review_body,reviews_split
1087812,this tarnished it is not sterling silver and t...,"[this, tarnished, it, is, not, sterling, silve..."
220841,got one for the most beautiful woman ive ever ...,"[got, one, for, the, most, beautiful, woman, i..."
1271914,when i received this ring i was so disappointe...,"[when, i, received, this, ring, i, was, so, di..."
387606,pretty but damage,"[pretty, but, damage]"
349019,the enamel on the ring scratches very easily a...,"[the, enamel, on, the, ring, scratches, very, ..."
...,...,...
1531270,i do not wear silver and very few 34dangle34 e...,"[i, do, not, wear, silver, and, very, few, 34d..."
176086,100 percent satisfied excellent product and ha...,"[100, percent, satisfied, excellent, product, ..."
1381040,the picture doesnt do this beautiful acelet j...,"[the, picture, doesnt, do, this, beautiful, ac..."
468354,this was a gift for my wife and she loved it,"[this, was, a, gift, for, my, wife, and, she, ..."


In [19]:
features=[]
for lis in X['reviews_split']:
    vector=np.zeros(300,)
    count=len(lis)
    for j in lis:
        if j in wv.key_to_index:
            vector=vector+wv[j]
    features.append(vector/count)
    
X['input']=features


/var/folders/3c/dvx5c3n50kx5fh6qcgyy221w0000gn/T/ipykernel_71284/2766180496.py:8: RuntimeWarning: invalid value encountered in true_divide
  features.append(vector/count)


In [20]:
df3 = X.input.apply(pd.Series)
df3=df3.replace(np.nan,0)
X_train,X_test,Y_train, Y_test = train_test_split(df3,Y, test_size=0.2, random_state=30)

**Perceptron model - accuracy obtained with TF-IDF features - 41%; Word2Vec - 33% . In case of perceptron model, the TF-IDF features perform better than word2Vec features but not by much**

In [21]:
from sklearn.linear_model import Perceptron
clf_perceptron = Perceptron()
clf_perceptron.fit(X_train, Y_train)

Perceptron()

In [22]:
y_test_perceptron=clf_perceptron.predict(X_test)

In [23]:
from sklearn.metrics import classification_report
report=classification_report(Y_test, y_test_perceptron,output_dict=True)

In [24]:
report

{'1': {'precision': 0.614221916867056,
  'recall': 0.4452191235059761,
  'f1-score': 0.5162407968817669,
  'support': 4016},
 '2': {'precision': 0.37142857142857144,
  'recall': 0.07454500124657193,
  'f1-score': 0.12416943521594685,
  'support': 4011},
 '3': {'precision': 0.31971153846153844,
  'recall': 0.09922904750062174,
  'f1-score': 0.1514518884038717,
  'support': 4021},
 '4': {'precision': 0.25334829254610475,
  'recall': 0.8992035838725734,
  'f1-score': 0.39531703047212646,
  'support': 4018},
 '5': {'precision': 0.7212903225806452,
  'recall': 0.14209456024402645,
  'f1-score': 0.23741771076661713,
  'support': 3934},
 'accuracy': 0.3329,
 'macro avg': {'precision': 0.4560001283767832,
  'recall': 0.3320582632739539,
  'f1-score': 0.2849193723480658,
  'support': 20000},
 'weighted avg': {'precision': 0.45487924413872244,
  'recall': 0.3329,
  'f1-score': 0.2851319895396592,
  'support': 20000}}

**SVM model - accuracy obtained with TF-IDF features - 49%; Word2Vec - 48%. Similar to case of perceptron model, In case of SVM model too, the TF-IDF features perform better than word2Vec features but by a very marginal difference .**

In [25]:
from sklearn.svm import LinearSVC
clf_SVM = LinearSVC()
clf_SVM.fit(X_train, Y_train)
y_test_SVM=clf_SVM.predict(X_test)
report_SVM=classification_report(Y_test, y_test_SVM,output_dict=True)

In [26]:
report_SVM

{'1': {'precision': 0.5076977526101575,
  'recall': 0.7143924302788844,
  'f1-score': 0.5935657391124445,
  'support': 4016},
 '2': {'precision': 0.38315318673127097,
  'recall': 0.256295188232361,
  'f1-score': 0.30714072303555423,
  'support': 4011},
 '3': {'precision': 0.40065952184666115,
  'recall': 0.36259636906242226,
  'f1-score': 0.3806788511749347,
  'support': 4021},
 '4': {'precision': 0.43756558237145854,
  'recall': 0.3113489298158288,
  'f1-score': 0.36382143376472303,
  'support': 4018},
 '5': {'precision': 0.5688854489164087,
  'recall': 0.7473309608540926,
  'f1-score': 0.6460118655240605,
  'support': 3934},
 'accuracy': 0.4773,
 'macro avg': {'precision': 0.45959229849519134,
  'recall': 0.4783927756487178,
  'f1-score': 0.4582437225223434,
  'support': 20000},
 'weighted avg': {'precision': 0.45914637049063084,
  'recall': 0.4773,
  'f1-score': 0.4574828154391955,
  'support': 20000}}

**Question 4a**

In [27]:
#Creating tensor data
#training data
xtrain_np=X_train.to_numpy()
x=torch.from_numpy(xtrain_np)

ytrain_np=Y_train.to_numpy()
y=torch.from_numpy(ytrain_np-1)

#testing data
xtest_np=X_test.to_numpy()
test_x=torch.from_numpy(xtest_np)

ytest_np=Y_test.to_numpy()
test_y=torch.from_numpy(ytest_np-1)

In [37]:
model=torch.nn.Sequential(
    torch.nn.Linear(300,50),
    torch.nn.ReLU(),
    torch.nn.Linear(50,10),
    torch.nn.ReLU(),
    torch.nn.Linear(10,5)    
)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [38]:
model=model.float()
for t in range(1000):
    y_pred = model(x.float())
    loss = criterion(y_pred, y)
    
    optimizer.zero_grad()
    loss.backward()
    
    optimizer.step()
    
    if(t%50==0):
        pred=model(x.float())
        _,predicted = torch.max(pred.data,1)
        correct = (predicted == y).sum()
        print(" traning Accuracy for",t,"-",correct/len(y))
    

 traning Accuracy for 0 - tensor(0.2008)
 traning Accuracy for 50 - tensor(0.2024)
 traning Accuracy for 100 - tensor(0.2779)
 traning Accuracy for 150 - tensor(0.3130)
 traning Accuracy for 200 - tensor(0.3367)
 traning Accuracy for 250 - tensor(0.3516)
 traning Accuracy for 300 - tensor(0.3626)
 traning Accuracy for 350 - tensor(0.3648)
 traning Accuracy for 400 - tensor(0.3715)
 traning Accuracy for 450 - tensor(0.3758)
 traning Accuracy for 500 - tensor(0.3847)
 traning Accuracy for 550 - tensor(0.3401)
 traning Accuracy for 600 - tensor(0.3665)
 traning Accuracy for 650 - tensor(0.3733)
 traning Accuracy for 700 - tensor(0.3796)
 traning Accuracy for 750 - tensor(0.3848)
 traning Accuracy for 800 - tensor(0.3900)
 traning Accuracy for 850 - tensor(0.3947)
 traning Accuracy for 900 - tensor(0.3992)
 traning Accuracy for 950 - tensor(0.4046)


In [39]:
pred=model(test_x.float())
_,predicted = torch.max(pred.data,1)
correct = (predicted == test_y).sum()
print("Accuracy for question 4a:",correct/len(test_y))

Accuracy for question 4a: tensor(0.3986)


**Question 4b**

In [24]:
#Creating feature vector

features=[]
for lis in X['reviews_split']:
    wordvecs=[]
    vector=np.zeros(300,)
    for j in range(len(lis)):
        if(j>=10):
            break
        else:
            if lis[j] in wv.key_to_index:
                wordvecs.append(wv[lis[j]])
    while len(wordvecs)<10:
        wordvecs.append(np.zeros(300,))
        
    features.append(wordvecs)
    
X['input_2']=features

In [25]:
df4 = X.input_2.apply(pd.Series)
df=pd.DataFrame()
for column in df4.columns:
    temp=df4.iloc[:,column].apply(pd.Series)
    df=pd.concat([df,temp],axis=1)
df

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
1087812,0.109375,0.140625,-0.031738,0.166016,-0.071289,0.015869,-0.003113,-0.084961,-0.048584,0.055664,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
220841,0.062012,0.108398,-0.096680,0.079102,0.033936,-0.347656,-0.069824,-0.052490,-0.132812,0.191406,...,-0.203125,-0.010986,-0.122559,0.157227,-0.045898,0.056641,-0.060547,-0.174805,-0.023071,-0.043457
1271914,0.170898,0.024292,0.138672,0.022217,0.068848,-0.090820,-0.031738,-0.133789,0.178711,0.156250,...,0.075684,0.225586,-0.111328,-0.039551,0.035156,-0.113281,-0.176758,0.028198,-0.177734,-0.006042
387606,0.123047,-0.046143,-0.202148,0.144531,-0.027100,-0.037598,-0.103027,-0.075195,0.194336,0.138672,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
349019,0.080078,0.104980,0.049805,0.053467,-0.067383,-0.120605,0.035156,-0.118652,0.043945,0.030151,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531270,-0.225586,-0.019531,0.090820,0.237305,-0.029297,0.093262,-0.058838,-0.041016,0.052246,0.020020,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
176086,0.081055,-0.235352,-0.045898,-0.036377,-0.063477,-0.130859,0.030396,-0.255859,0.484375,-0.049561,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1381040,0.080078,0.104980,0.049805,0.053467,-0.067383,-0.120605,0.035156,-0.118652,0.043945,0.030151,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
468354,0.109375,0.140625,-0.031738,0.166016,-0.071289,0.015869,-0.003113,-0.084961,-0.048584,0.055664,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
X_train,X_test,Y_train, Y_test = train_test_split(df,Y, test_size=0.2, random_state=30)

In [27]:
#training data
xtrain_np=X_train.to_numpy()
x=torch.from_numpy(xtrain_np)

ytrain_np=Y_train.to_numpy()
y=torch.from_numpy(ytrain_np-1)

#testing data
xtest_np=X_test.to_numpy()
test_x=torch.from_numpy(xtest_np)

ytest_np=Y_test.to_numpy()
test_y=torch.from_numpy(ytest_np-1)


In [29]:
model=torch.nn.Sequential(
    torch.nn.Linear(3000,50),
    torch.nn.ReLU(),
    torch.nn.Linear(50,10),
    torch.nn.ReLU(),
    torch.nn.Linear(10,5)    
)

In [32]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [33]:
model=model.float()
for t in range(150):
    y_pred=model(x.float())
    loss = criterion(y_pred, y)
    
    optimizer.zero_grad()
    loss.backward()
    
    optimizer.step()
    
    if(t%10==0):
        pred=model(x.float())
        _,predicted = torch.max(pred.data,1)
        correct = (predicted == y).sum()
        print(" train Accuracy for",t,"-",correct/len(y))

 train Accuracy for 0 - tensor(0.2102)
 train Accuracy for 10 - tensor(0.2211)
 train Accuracy for 20 - tensor(0.2332)
 train Accuracy for 30 - tensor(0.2463)
 train Accuracy for 40 - tensor(0.2625)
 train Accuracy for 50 - tensor(0.2799)
 train Accuracy for 60 - tensor(0.2923)
 train Accuracy for 70 - tensor(0.3007)
 train Accuracy for 80 - tensor(0.3083)
 train Accuracy for 90 - tensor(0.3137)
 train Accuracy for 100 - tensor(0.3202)
 train Accuracy for 110 - tensor(0.3245)
 train Accuracy for 120 - tensor(0.3263)
 train Accuracy for 130 - tensor(0.3288)
 train Accuracy for 140 - tensor(0.3307)


In [34]:
pred=model(test_x.float())

In [35]:
_,predicted = torch.max(pred.data,1)
correct = (predicted == test_y).sum()
print("Test Accuracy - 4b:",correct/len(test_y))

Test Accuracy - 4b: tensor(0.3249)
